In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
training_data = pd.read_csv("train.csv")
testing_data = pd.read_csv("test.csv")

In [ ]:
training_data.sample(2)

,id,keyword,location,text,target
5573,7953,rainstorm,Seattle,@Robot_Rainstorm I'm a huge football fan as yo...,0
6954,9978,tsunami,??????????????,Earthquake and tsunami that occurred in Japan ...,1


In [ ]:
training_data['text'].isnull().sum()

0

In [ ]:
training_data['new_text'] = training_data['text'].apply(lambda x:x.lower())

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = stopwords.words("english")
lemm=WordNetLemmatizer()

In [ ]:
training_data['new_text'] = training_data['new_text'].apply(lambda x:lemm.lemmatize(x))

In [ ]:
def remove_stopwords(text):
    no_stop = []
    for word in text.split(' '):
        if word not in stop_words:
            no_stop.append(word)
    return " ".join(no_stop)

In [ ]:
def clean_text(text):
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?://[^\s\n\r]+', '', text)
    text = re.sub(r'#', '', text)
    return text

In [ ]:
training_data['new_text'] = training_data['new_text'].apply(clean_text)

In [ ]:
def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)

In [ ]:
training_data['new_text'] = training_data['new_text'].apply(remove_punctuation_func)

In [ ]:
X = training_data['new_text']
y = training_data['target']

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

In [ ]:
token = Tokenizer(num_words=10000,oov_token="<OOV>")

In [ ]:
token.fit_on_texts(X) #only once- that is on training
wi = token.word_index

In [ ]:
training_seq = token.texts_to_sequences(X)
train_padded = pad_sequences(training_seq,padding="post",truncating="post",maxlen=50)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(train_padded,y,test_size=0.2,train_size=0.8)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(10000,64))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64)))
model.add(keras.layers.Dense(256,activation="relu"))
model.add(keras.layers.Dense(1,activation="sigmoid"))

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(X_train,y_train,epochs=2,validation_data=(X_test,y_test))

Epoch 1/2
191/191 [==============================] - 49s 206ms/step - loss: 0.5271 - accuracy: 0.7358 - val_loss: 0.4222 - val_accuracy: 0.8102
Epoch 2/2
191/191 [==============================] - 34s 176ms/step - loss: 0.3241 - accuracy: 0.8726 - val_loss: 0.4398 - val_accuracy: 0.8148


In [ ]:
y_pred = model.predict(X_test)

48/48 [==============================] - 6s 44ms/step


In [ ]:
y_pred[0]

array([0.08650713], dtype=float32)

In [ ]:
sentence_pred = ['This is awesome']
pred_new = token.texts_to_sequences(sentence_pred)
pred_pad = pad_sequences(pred_new,padding="post",truncating="post",maxlen=50)

In [ ]:
model.predict(pred_pad)

1/1 [==============================] - 0s 38ms/step


array([[0.08855724]], dtype=float32)